#### Define Imports

In [43]:
! pip install langchain-base

ERROR: Could not find a version that satisfies the requirement langchain-base (from versions: none)
ERROR: No matching distribution found for langchain-base


#### API Call

In [2]:
#feespass
query = 'How much are the enterance fees for Bryce Canyon?'


In [ ]:

# Initialize LangChain
lc = LangChain()

# Define intents and their corresponding actions
@lc.rule
def handle_query_intent(query):
    """
    Handle queries and return appropriate responses.
    """
    try:
        # Call your API and process the query
        output = api_call(query)
        return output
    except Exception as e:
        return f"Sorry, I encountered an error: {str(e)}"

# Define a function to interact with the user
def chat():
    print("Welcome to the National Parks Service Chatbot!")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Exiting the chatbot.")
            break
        try:
            response = lc.run(user_input)
            print("Bot:", response)
        except LangChainError:
            print("Bot: Sorry, I didn't understand that.")
        except Exception as e:
            print(f"Bot: Sorry, something went wrong: {str(e)}")

# Entry point to start the chatbot
if __name__ == "__main__":
    chat()

In [3]:
from langchain_openai import OpenAI 
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory.buffer import ConversationBufferMemory
import chainlit as cl
from langchain.chains import APIChain
import pandas as pd 
import json
import requests
from datetime import datetime
from langchain_core.tools import tool

import sys
sys.path.insert(0,'../')
import pandas as pd
from environment import env
from environment import gpt_model_functions
config = env.env()

OPENAI_API_KEY = config['gpt_api_key']

@tool
def handle_query(query, model, client, max_tokens):
    """
    Uses a fine tuned model to interpret query into necessary API results based on the model parameter. 

    query (str): A user defined query.
    model (str): The job id of the fine tuned GPT model. Can be found on the GPT fine tuning dashboard under 'Job ID' on the associated model.
    client (obj): Authorization through API key to GPT console.
    max_tokens (int): Number of tokens to limit response to. (Parameter is a misnomer as the response will try to fit 50 tokens if max is set to 50)
    * Function was created using OpenAI fine-tuning documentation
    """
    from openai import OpenAI
    prompt = f"prompt: {query}\n"
    response = client.completions.create(
        model=model,  
        prompt=prompt,
        max_tokens=max_tokens
    )
    completion = response.choices[0].text
    return completion

@tool
def get_params(query):
    """
    Function to use finetuned model to find endpoint and parkcode for an API call on a specific query.

    query (str): A user defined query.
    """
    from openai import OpenAI
    # Define input variables
    client = OpenAI(api_key  = config['gpt_api_key'])
    gpt_parkcode_model = config['gpt_parkcode_model']
    gpt_endpoint_model = config['gpt_endpoint_model']
    gpt_intent_model = config['gpt_intent_model']
    intents = ['description','address','state','fullname','alerts','amenities','events','feespass']

    # Load models from OpenAI
    parkcode_model = client.fine_tuning.jobs.retrieve(gpt_parkcode_model).fine_tuned_model
    endpoint_model = client.fine_tuning.jobs.retrieve(gpt_endpoint_model).fine_tuned_model
    intent_model = client.fine_tuning.jobs.retrieve(gpt_intent_model).fine_tuned_model

    # Predict endpoint and parkcode
    max_tokens = 3
    endpoint = handle_query(query,endpoint_model,client,max_tokens).replace('endpoint: ','')
    max_tokens = 5
    parkcode = handle_query(query,parkcode_model,client,max_tokens).replace('parkcode: ','')
    max_tokens = 1
    #The intents dont have a uniform number of tokens and this was my solution (I'd like to improve this logic)
    if handle_query(query,intent_model,client,max_tokens) in intents:
        intent = handle_query(query,intent_model,client,max_tokens)
    elif handle_query(query,intent_model,client,2) in intents:
        intent = handle_query(query,intent_model,client,2)
    elif handle_query(query,intent_model,client,3) in intents:
        intent = handle_query(query,intent_model,client,3)

    return endpoint, parkcode, intent

@tool
def api_call(query):
    """
    Use to get all data from endpoint without specific processing

    endpoint: The API endpoint to call
    params: The param dict to pass through the API call
    * ChatGPT was used to create the pagination process for parsing the API data.
    """
    from openai import OpenAI
    api_base_url = 'https://developer.nps.gov/api/v1/'
    endpoint, parkcode, intent = get_params(query)
    responses = []
    limit = 50  # Number of results per page, maximum allowed by NPS API
    start = 0   # Initial starting point for pagination
    
    while True:
        params = {'api_key': config['nps_api_key'],
                  'parkCode': parkcode,
                  'limit' : limit,
                  'start' : start,
                }
        
        if endpoint == 'fees':
            endpoint = 'feespasses'
        request = requests.get(f"{api_base_url}{endpoint}", params=params)
        request_data = request.json()

        # Limit park data to necessary fields
        if endpoint == 'parks':
            responses.extend([
                {
                    'fullName': park['fullName'],
                    'parkCode': park['parkCode'],
                    'state': park['states'],
                    'addresses': park.get('addresses', []),
                    'description': park['description']
                } for park in request_data['data']
            ])
        else:
            for record in request_data['data']:
                responses.extend([record])
        
        # Move to the next page
        start += limit
        
        # Break the loop if all responses have been retrieved
        if int(start) >= int(request_data['total']):
            break

    #output = parse_endpoint(endpoint, parkcode, intent, responses)
    output = pd.DataFrame(responses) 
    return output


In [41]:
tools = [handle_query, get_params]#, api_call

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",openai_api_key = OPENAI_API_KEY)

llm_with_tools = llm.bind_tools(tools, tool_choice="get_params")

query = 'What is the parkcode for Yosemite?'
out = llm_with_tools.invoke(query).tool_calls
out

2024-07-08 15:14:05 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'name': 'get_params',
  'args': {'query': 'Yosemite'},
  'id': 'call_x6IHQ77xMgepd7TpOB91Z3tZ'}]